# Mean Shift and Cam Shift
Mean Shift and Cam Shift are similar techniques but yield different outcomes. Mean Shift provides simpler results; however, it struggles with handling rotations and adapting to changes in object size. In contrast, Cam Shift is more advanced and flexible, capable of managing rotations, size variations, and more.

## 1. Mean Shift
Mean Shift is a non-parametric algorithm used for clustering and mode-seeking(finding the most frequently occurring value in a dataset) in data. In object tracking, Mean Shift tracks an object by repeatedly adjusting a window to follow the object’s color distribution.

![image](https://miro.medium.com/v2/resize:fit:424/format:webp/1*FyVns5w5JB4UApIlnx625g.gif)

It is simple and computationally efficient but it can not handle rotations and varying sizes. When computational resources are limited, Mean Shift can be considered for use instead of the Cam Shift algorithm.

## 2. Cam Shift
Cam shift is an advanced version of Mean shift, it applies Mean shift first. It adjusts the tracking window size and orientation to follow objects that rotate or change size. It can detect objects in more complex scenarios.

It requires more computational resources when compared with Mean Shift.

![image](https://miro.medium.com/v2/resize:fit:640/format:webp/1*UkqFIOohadZb_gxT6k9O_A.gif)

## Object Tracker with Cam shift and Mean shift
Now it is time to create an object detector using these 2 algorithms. The main idea is quite simple: First user draws a rectangle to the interested area (object) with a mouse right-click and presses the ‘ESC’ key. After that, a new window appears, and the object is tracked within that window.

### STEP 1: The user defines the object by drawing a rectangle to the first frame of the video, the user presses the mouse right button for the first point pair and the same button for the second point pair.

### STEP 2: The program detects the color of the object that the user selected.

### STEP 3: Tracking object.


### Implementation

In [1]:
import cv2
import numpy as np

In [4]:
import cv2
import numpy as np

# Path to video
video_path = r"./videos/video.mp4"
video = cv2.VideoCapture(video_path)

# Read the first frame
ret, frame = video.read()
if not ret:
    print("Error: Could not read video.")
    exit()

# Initialize variables
x_min, y_min, x_max, y_max = 0, 0, 0, 0
drawing = False  # Flag to check if drawing is in progress
temp_frame = frame.copy()


# Function to select ROI using left-click dragging
def coordinate_chooser(event, x, y, flags, param):
    global x_min, y_min, x_max, y_max, drawing, frame, temp_frame

    if event == cv2.EVENT_LBUTTONDOWN:  # Start selecting
        drawing = True
        x_min, y_min = x, y
        x_max, y_max = x, y  # Initialize end coordinates

    elif event == cv2.EVENT_MOUSEMOVE and drawing:  # Update rectangle while dragging
        temp_frame = frame.copy()
        x_max, y_max = x, y
        cv2.rectangle(temp_frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.imshow("coordinate_screen", temp_frame)

    elif event == cv2.EVENT_LBUTTONUP:  # Finalize selection
        drawing = False
        x_max, y_max = x, y
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.imshow("coordinate_screen", frame)


# Set up window and mouse callback
cv2.namedWindow("coordinate_screen")
cv2.setMouseCallback("coordinate_screen", coordinate_chooser)

# Show first frame for ROI selection
while True:
    cv2.imshow("coordinate_screen", frame)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:  # Press ESC to continue
        break

cv2.destroyAllWindows()

# Extract the selected ROI
object_image = frame[y_min:y_max, x_min:x_max]
if object_image.size == 0:
    print("Error: No valid ROI selected.")
    exit()

# Convert ROI to HSV and get the center pixel hue
hsv_object = cv2.cvtColor(object_image, cv2.COLOR_BGR2HSV)
height, width, _ = hsv_object.shape
cx, cy = width // 2, height // 2
hue_value = hsv_object[cy, cx, 0]  # Extract hue

# Determine the detected color based on hue value
color = "unknown"
if hue_value < 5 or hue_value >= 170:
    color = "red"
elif hue_value < 22:
    color = "orange"
elif hue_value < 33:
    color = "yellow"
elif hue_value < 78:
    color = "green"
elif hue_value < 131:
    color = "blue"
elif hue_value < 170:
    color = "violet"

# Hue value dictionary for color range
hue_dict = {
    "red": [[0, 100, 100], [10, 255, 255]],
    "orange": [[10, 100, 100], [20, 255, 255]],
    "yellow": [[20, 100, 100], [30, 255, 255]],
    "green": [[50, 100, 100], [70, 255, 255]],
    "blue": [[110, 50, 50], [130, 255, 255]],
    "violet": [[140, 50, 50], [170, 255, 255]]
}

# Get color bounds
lower_bound = np.array(hue_dict[color][0])
upper_bound = np.array(hue_dict[color][1])
print(f"Detected color: {color}")

# Reopen video to start tracking
video = cv2.VideoCapture(video_path)

# Read first frame again for tracking setup
ret, frame = video.read()
if not ret:
    print("Error: Could not read video for tracking.")
    exit()

# Setup tracking window
track_window = (x_min, y_min, x_max - x_min, y_max - y_min)

# Extract ROI from the first frame
roi = frame[y_min:y_max, x_min:x_max]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Create mask and calculate histogram
mask = cv2.inRange(hsv_roi, lower_bound, upper_bound)
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Mean-Shift termination criteria
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# Tracking loop
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute back projection
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

    # Apply Mean-Shift tracking
    ret, track_window = cv2.meanShift(dst, track_window, term_crit)
    x, y, w, h = track_window

    # Draw tracking rectangle
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Show tracking result
    cv2.putText(frame, f"Detected color: {color}", (25, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Object Tracking", frame)

    # Press ESC to exit
    if cv2.waitKey(30) & 0xFF == 27:
        break

# Release resources
video.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is n

Detected color: red


QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is not the object's thread (0x23a6ec60).
Cannot move to target thread (0x23791e40)

QObject::moveToThread: Current thread (0x23791e40) is n